In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
import pickle

In [2]:
m=pd.read_csv(r"C:\Users\Acer\OneDrive\Desktop\College\Projects\Movie Recommendation\tmdb_5000_movies.csv")
c=pd.read_csv(r"C:\Users\Acer\OneDrive\Desktop\College\Projects\Movie Recommendation\tmdb_5000_credits.csv")

movies=m.merge(c, on='title') #merge dataset on the basis of Title of the mvoies
print(movies.shape)
print(movies.head(1))

(4809, 23)
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                      homepage     id  \
0  http://www.avatarmovie.com/  19995   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   

  original_title                                           overview  \
0         Avatar  In the 22nd century, a paraplegic Marine is di...   

   popularity                               production_companies  ... runtime  \
0  150.437577  [{"name": "Ingenious Film Partners", "id": 289...  ...   162.0   

                                    spoken_languages    status  \
0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   

                       tagline   title vote_average vote_count movie_id  \
0  Enter the World of Pandora.  Avatar          7.2      11800    19995   

                                

In [3]:
print(movies.columns)
movies=movies[['id','title','overview','keywords','genres','cast','crew']] #selected coloumns for recommendation
print(movies.head(2))

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')
      id                                     title  \
0  19995                                    Avatar   
1    285  Pirates of the Caribbean: At World's End   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   

                                            keywords  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...   

                                              genres  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  [{"id": 12, "name

In [4]:
# print(movies.isnull().sum())
movies.dropna(inplace=True) # drop the empty rows
print(movies.isnull().sum())

movies.duplicated().sum()

id          0
title       0
overview    0
keywords    0
genres      0
cast        0
crew        0
dtype: int64


0

In [5]:
movies['genres'].iloc[0]
#desired format: [Action,Adventure,Science Fiction]


'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [6]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):  # literal_eval converts string to list
        L.append(i['name'])
    return L


In [7]:
movies['genres']=movies['genres'].apply(convert) #apply function is used to apply a function on a coloumn of a dataset

In [8]:
movies.head(1)

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[Action, Adventure, Fantasy, Science Fiction]","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [9]:
movies['keywords']=movies['keywords'].apply(convert)

In [10]:
movies.head(1)

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [11]:
def convert1(obj):
    L = []
    counter=0
    for i in ast.literal_eval(obj): 
        if counter!=3:          # as we only need the top 3 casts
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [12]:
movies['cast']=movies['cast'].apply(convert1)

In [13]:
movies.head(2)

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [14]:
movies['crew'].iloc[0] # we only need the director of the movie

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [15]:
def director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director': #as we only need the director from the crew coloumn
            L.append(i['name'])
            break
    return L

In [16]:
movies['crew']=movies['crew'].apply(director)

In [17]:
movies.head()

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [18]:
# As all other coloumns are in list, convert overview coloumn from string to list for easier processing

movies['overview']=movies['overview'].apply(lambda x:x.split()) # lambda function takes x input and coverts it to list

In [19]:
movies.head()

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [20]:
# we will join the spaces between all the attributes except id and title for avoiding confusion on the basis of names
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['overview']=movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [21]:
movies.head()

,id,title,overview,keywords,genres,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drugabuse, exoticisland, eastindiatrad...","[Adventure, Fantasy, Action]","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[Action, Adventure, Crime]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[dccomics, crimefighter, terrorist, secretiden...","[Action, Crime, Drama, Thriller]","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[basedonnovel, mars, medallion, spacetravel, p...","[Action, Adventure, ScienceFiction]","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [22]:
# concatenate all 5 coloumns into a single coloumn "tags"
movies['tags']=movies['genres']+movies['overview']+movies['keywords']+movies['cast']+movies['crew']
movies.head()

,id,title,overview,keywords,genres,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...","[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[Action, Adventure, Fantasy, ScienceFiction, I..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[ocean, drugabuse, exoticisland, eastindiatrad...","[Adventure, Fantasy, Action]","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Adventure, Fantasy, Action, Captain, Barbossa..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[spy, basedonnovel, secretagent, sequel, mi6, ...","[Action, Adventure, Crime]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[Action, Adventure, Crime, A, cryptic, message..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[dccomics, crimefighter, terrorist, secretiden...","[Action, Crime, Drama, Thriller]","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Action, Crime, Drama, Thriller, Following, th..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[basedonnovel, mars, medallion, spacetravel, p...","[Action, Adventure, ScienceFiction]","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[Action, Adventure, ScienceFiction, John, Cart..."


In [23]:
# As we have concatenated all the coloumns into tags we do not need them individually so, update dataset
movies1=movies[['id','title','tags']]
movies1.head()

,id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction, I..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, Captain, Barbossa..."
2,206647,Spectre,"[Action, Adventure, Crime, A, cryptic, message..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, Following, th..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction, John, Cart..."


In [24]:
#Convert tags to string
movies1['tags']=movies1['tags'].apply(lambda x: " ".join(x))
movies1.head()

C:\Users\Acer\AppData\Local\Temp\ipykernel_3104\3724224147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies1['tags']=movies1['tags'].apply(lambda x: " ".join(x))


,id,title,tags
0,19995,Avatar,Action Adventure Fantasy ScienceFiction In the...
1,285,Pirates of the Caribbean: At World's End,"Adventure Fantasy Action Captain Barbossa, lon..."
2,206647,Spectre,Action Adventure Crime A cryptic message from ...
3,49026,The Dark Knight Rises,Action Crime Drama Thriller Following the deat...
4,49529,John Carter,Action Adventure ScienceFiction John Carter is...


In [25]:
movies1['tags'].iloc[0]

'Action Adventure Fantasy ScienceFiction In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [26]:
#Covert tags coloumn into lower case as it is recommended
movies1['tags']=movies1['tags'].apply(lambda x: x.lower())


C:\Users\Acer\AppData\Local\Temp\ipykernel_3104\2718806425.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies1['tags']=movies1['tags'].apply(lambda x: x.lower())


In [27]:
movies1.head(2)

,id,title,tags
0,19995,Avatar,action adventure fantasy sciencefiction in the...
1,285,Pirates of the Caribbean: At World's End,"adventure fantasy action captain barbossa, lon..."


In [28]:
movies1['tags'].iloc[1]

"adventure fantasy action captain barbossa, long believed to be dead, has come back to life and is headed to the edge of the earth with will turner and elizabeth swann. but nothing is quite as it seems. ocean drugabuse exoticisland eastindiatradingcompany loveofone'slife traitor shipwreck strongwoman ship alliance calypso afterlife fighter pirate swashbuckler aftercreditsstinger johnnydepp orlandobloom keiraknightley goreverbinski"

In [29]:
# convert the tags coloumn into an vector

In [30]:
vectorizer=TfidfVectorizer()
tag_vectors=vectorizer.fit_transform(movies1['tags']).toarray() #transforms into numerical value

In [31]:
print(tag_vectors)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
print(len(vectorizer.get_feature_names_out()))

35611


In [33]:
# Deriving similarity between vectors(movies) by angle between them
similarity=cosine_similarity(tag_vectors)

In [34]:
similarity #similarity with movie itself will be 1

array([[1.        , 0.02830149, 0.01718286, ..., 0.02063096, 0.01344477,
        0.0072699 ],
       [0.02830149, 1.        , 0.03134695, ..., 0.02802018, 0.0198414 ,
        0.01200361],
       [0.01718286, 0.03134695, 1.        , ..., 0.02927459, 0.01334713,
        0.01309662],
       ...,
       [0.02063096, 0.02802018, 0.02927459, ..., 1.        , 0.04181678,
        0.03079815],
       [0.01344477, 0.0198414 , 0.01334713, ..., 0.04181678, 1.        ,
        0.03931743],
       [0.0072699 , 0.01200361, 0.01309662, ..., 0.03079815, 0.03931743,
        1.        ]])

In [35]:
#List of all the movies
list_of_all_movies=movies['title'].tolist()

In [36]:
print(list_of_all_movies)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [37]:
if "Lights Out" in list_of_all_movies:
    print("Yes")
else:
    print("No")

Yes


In [38]:
def recommend(movie):
        movie_index= movies1[movies1['title']==movie].index[0]
        distances=similarity[movie_index]  
        movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
        
        for i in movies_list:
            print(movies1.iloc[i[0]].title)
               

In [39]:
movie=input("Enter your last seen movie:")
if movie in list_of_all_movies:
    print("Recommended movies:")
    recommend(movie)
else:
    print("Try Another movie")

Enter your last seen movie:Inception
Recommended movies:
Don Jon
Blood and Wine
Looper
Hesher
Tinker Tailor Soldier Spy


In [41]:
pickle.dump(movies1,open('movies.pkl','wb'))

In [42]:
pickle.dump(list_of_all_movies,open('movies_list.pkl','wb'))

In [43]:
pickle.dump(similarity,open('similarity.pkl','wb'))